#### 用线性分类模型，可能得出很多种不同的线性方程，对数据集的划分方式有很多种，那如何去选择一个具有最大容忍度的分类模型呢？
#### 支持向量机Support Vector Marchine便是更具训练样本的分布，搜索所有线性分类中容忍度最佳的哪个。
#### 什么是支持向量呢？我们把可以用来真正帮助决策最优线性模型的数据点叫“支持向量”。逻辑回归在训练过程中考虑了所有样本对参数的影响
#### 训练耗时且不一定得出最佳的分类器，支持向量机只选取少数的样本来获取最佳的分类器，训练速度更快、线性模型容忍度更高、泛化能力更强

## 使用支持向量机，实现手写数字分类。

#### 1.导入数据

In [2]:
from sklearn.datasets import load_digits
#调用load_digits方法将数据载入df中
df = load_digits()

In [3]:
#查看数据说明
print(df.DESCR)

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [4]:
df.data.shape

(1797, 64)

In [5]:
#每张图片使用一个8*8的矩阵表示，但是这里把二维矩阵拼接成了一个一维的向量，虽然损失了数据的结构信息，但是并不影响我们的分类任务
df.data[0]

array([  0.,   0.,   5.,  13.,   9.,   1.,   0.,   0.,   0.,   0.,  13.,
        15.,  10.,  15.,   5.,   0.,   0.,   3.,  15.,   2.,   0.,  11.,
         8.,   0.,   0.,   4.,  12.,   0.,   0.,   8.,   8.,   0.,   0.,
         5.,   8.,   0.,   0.,   9.,   8.,   0.,   0.,   4.,  11.,   0.,
         1.,  12.,   7.,   0.,   0.,   2.,  14.,   5.,  10.,  12.,   0.,
         0.,   0.,   0.,   6.,  13.,  10.,   0.,   0.,   0.])

#### 2.分割数据，用于测试和训练。75%的数据用于训练，25%的数据用于测试

In [6]:
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.data,df.target,test_size=0.25,random_state=123)

D:\softwares\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
y_train.shape

(1347,)

In [8]:
y_test.shape

(450,)

#### 3.使用基于线性假设的支持向量机，进行手写字体的分类

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

#对测试数据和训练数据标准化
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test  = ss.transform(x_test)

#初始化SVC模型，使用linear核函数
#【核函数的作用就是隐含着一个从低维空间到高维空间的映射，而这个映射可以把低维空间中线性不可分的两类点变成线性可分的】
#有很多种核函数，linear、poly、rbf等
svc = SVC(kernel='linear')

#训练模型
svc.fit(x_train,y_train)

#预测
svc_predict = svc.predict(x_test)



#### 4.算法性能评估

In [13]:
#使用Accuary，精确率，召回率，F1指标来衡量算法的性能
print("svc 手写字体识别准确率为：",svc.score(x_test,y_test))

svc 手写字体识别准确率为： 0.971111111111


In [14]:
#使用classification_report方法度量算法性能
from sklearn.metrics import classification_report
print("svc 手写字体性能指标：\n",classification_report(y_test,svc_predict,target_names=df.target_names.astype(str)))

svc 手写字体性能指标：
              precision    recall  f1-score   support

          0       1.00      1.00      1.00        51
          1       0.89      0.98      0.93        42
          2       1.00      1.00      1.00        41
          3       1.00      1.00      1.00        40
          4       0.98      1.00      0.99        52
          5       0.98      0.92      0.95        48
          6       1.00      0.96      0.98        48
          7       1.00      1.00      1.00        41
          8       0.95      0.91      0.93        45
          9       0.91      0.95      0.93        42

avg / total       0.97      0.97      0.97       450



#### 注意：召回率、精确率、F1指标适用于二分类任务，对于多分类任务，是无法直接计算上述指标的。通常的做法是逐一评估某个类别的三个性能指标
####     把其他的指标看成是负样本来计算，这样就创建了10个二分类任务。结果如上所示。是一个10行的混淆矩阵

### 5.svm算法特点分析

#### a.精妙的模型假设，可以在海量高维的数据中筛选对任务最为有效的训练样本。这样做得好处是节省模型学习所需要的数据内存，同时还会提供算法最终的预测性能
##### b.弊是多出来了计算的开销